<a href="https://colab.research.google.com/github/mehdi780311/neural-network-for-occupational-disease-prediction/blob/main/my_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.وارد کردن کتابخانه‌های لازم

In [73]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.layers import Input
import matplotlib.pyplot as plt

2. ساخت یک مجموعه داده نمونه با پانداز
(در یک پروژه واقعی، این داده‌ها معمولا از یک فایل اکسل یا CSV خوانده می‌شود)

In [65]:
data = {
    'age': [25, 45, 38, 52, np.nan, 29], # np.nan یک مقدار گمشده است
    'job_title': ['joshkar', 'nasaaj', 'kargar_madan', 'joshkar', 'nasaaj', 'kargar_madan'],
    'exposure_level': [0.8, 1.2, 2.5, 0.9, 1.5, 2.8],
    'has_disease': [0, 1, 1, 0, 1, 1] # 0 = سالم, 1 = بیمار
}
df = pd.DataFrame(data)

print("--- داده‌های خام اولیه ---")
print(df)
print("\n" + "="*30 + "\n")

--- داده‌های خام اولیه ---
    age     job_title  exposure_level  has_disease
0  25.0       joshkar             0.8            0
1  45.0        nasaaj             1.2            1
2  38.0  kargar_madan             2.5            1
3  52.0       joshkar             0.9            0
4   NaN        nasaaj             1.5            1
5  29.0  kargar_madan             2.8            1




**شروع مراحل پیش‌پردازش **

3. مدیریت مقادیر گمشده: سن خالی را با میانگین سن بقیه پر می‌کنیم

In [37]:
mean_age = df['age'].mean()
df['age'].fillna(mean_age, inplace=True)
df['age'] = df['age'].astype(int)

/tmp/ipython-input-913701135.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['age'].fillna(mean_age, inplace=True)


4. کدگذاری داده‌های متنی: ستون شغل را به ستون‌های عددی تبدیل می‌کنیم

In [52]:
df_encoded = pd.get_dummies(df, columns=['job_title'])

print("--- داده‌های تمیز و کدگذاری شده ---")
print(df_encoded)
print("\n" + "="*30 + "\n")

--- داده‌های تمیز و کدگذاری شده ---
   age  exposure_level  has_disease  job_title_joshkar  \
0   25             0.8            0               True   
1   45             1.2            1              False   
2   38             2.5            1              False   
3   52             0.9            0               True   
4   37             1.5            1              False   
5   29             2.8            1              False   

   job_title_kargar_madan  job_title_nasaaj  
0                   False             False  
1                   False              True  
2                    True             False  
3                   False             False  
4                   False              True  
5                    True             False  




5. جدا کردن ویژگی‌ها (X) از ستون هدف (y)

In [68]:
X = df_encoded.drop('has_disease', axis=1)
y = df_encoded['has_disease']
#print(X)
print(y)

0    0
1    1
2    1
3    0
4    1
5    1
Name: has_disease, dtype: int64


6. مقیاس‌بندی ویژگی‌ها (X) در بازه 0 تا 1

In [70]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
print(X_scaled)

[[0.         0.         1.         0.         0.        ]
 [0.74074074 0.2        0.         0.         1.        ]
 [0.48148148 0.85       0.         1.         0.        ]
 [1.         0.05       1.         0.         0.        ]
 [0.44444444 0.35       0.         0.         1.        ]
 [0.14814815 1.         0.         1.         0.        ]]


 7. تقسیم نهایی داده به دو بخش آموزشی و آزمایشی
70% برای آموزش، 30% برای تست

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)
print(f"تعداد نمونه‌های آموزشی: {X_train.shape[0]}")
print(f"تعداد نمونه‌های آزمایشی: {X_test.shape[0]}")

تعداد نمونه‌های آموزشی: 4
تعداد نمونه‌های آزمایشی: 2


 ساخت یک مدل ترتیبی (Sequential) خالی
 این مدل در حال حاضر مثل یک ظرف خالی است که باید لایه‌ها را به آن اضافه کنیم

In [56]:
model = keras.Sequential()
print("مدل Sequential با موفقیت ساخته شد! این مدل فعلاً خالی است.")

مدل Sequential با موفقیت ساخته شد! این مدل فعلاً خالی است.


تعداد ویژگی‌های ورودی ما (تعداد ستون‌های داده‌های X)
این پارامتر فقط برای لایه اول لازم است

In [57]:
input_dim = X_train.shape[1]

 اضافه کردن لایه ورودی و اولین لایه پنهان

In [58]:
model.add(layers.Dense(units=16, activation='relu', input_shape=(input_dim,)))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# اضافه کردن یک لایه پنهان دیگر


In [59]:
model.add(layers.Dense(units=8, activation='relu'))


# اضافه کردن لایه خروجی
# چون پیش‌بینی ما دو حالته است (بیمار یا سالم)، از 1 نورون با تابع sigmoid استفاده می‌کنیم

In [60]:
model.add(layers.Dense(units=1, activation='sigmoid'))

# نمایش خلاصه‌ای از معماری مدل ساخته شده


In [61]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                │ (None, 16)             │            96 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 241 (964.00 B)

 Trainable params: 241 (964.00 B)

 Non-trainable params: 0 (0.00 B)

In [62]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

print("مدل با موفقیت کامپایل شد و آماده آموزش است!")

مدل با موفقیت کامپایل شد و آماده آموزش است!


In [63]:
print("--- شروع فرآیند آموزش مدل ---")

# آموزش مدل با داده‌های آموزشی
# ما به مدل می‌گوییم ۲۰ بار کل داده‌ها را مرور کن
history = model.fit(X_train, y_train,
                    epochs=20,
                    batch_size=1) # در هر مرحله ۱ نمونه را ببین (چون داده ما کوچک است)


print("\nآموزش مدل به پایان رسید!")

--- شروع فرآیند آموزش مدل ---
Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5333 - loss: 0.6906   
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9000 - loss: 0.6213 
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5333 - loss: 0.6856     
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7333 - loss: 0.6378 
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8333 - loss: 0.6224 
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8333 - loss: 0.5986 
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9000 - loss: 0.5951 
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9000 - loss: 0.5888 
Epoch 9/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5333 - loss: 0.6639     
Epoch 10/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7333 - loss: 0.6018
Epoch 11/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8333 - loss: 0.5888 
Epoch 12/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/s

In [64]:
print("\n--- ارزیابی مدل روی داده‌های آزمایشی ---")

# ارزیابی loss و accuracy روی مجموعه تست
loss, accuracy = model.evaluate(X_test, y_test)

print(f"\nدقت نهایی مدل روی داده‌های تست: {accuracy * 100:.2f}%")


--- ارزیابی مدل روی داده‌های آزمایشی ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step - accuracy: 0.5000 - loss: 0.7312

دقت نهایی مدل روی داده‌های تست: 50.00%


In [72]:
# فرض کنید اطلاعات کارگر جدید به این صورت است:
# سن: 40, شغل: جوشکار, سطح مواجهه: 1.1

# 1. تبدیل به فرمت عددی و کدگذاری شده بر اساس ستون‌های قبلی
# (فرض می‌کنیم ستون‌ها به ترتیب سن، سطح مواجهه، شغل جوشکار ::، شغل معدن و شغل نساج بودند)
new_worker_data = np.array([[40.0, 1.1, 1.0, 0.0,0.0]])

# 2. داده جدید را هم باید با همان scaler قبلی مقیاس‌بندی کنیم
new_worker_scaled = scaler.transform(new_worker_data)

# 3. انجام پیش‌بینی
prediction = model.predict(new_worker_scaled)
probability = prediction[0][0]

print(f"\nخروجی خام مدل (احتمال بیماری): {probability:.4f}")

# 4. تفسیر نتیجه
if probability > 0.5:
    print("نتیجه پیش‌بینی: این کارگر در معرض خطر بیماری قرار دارد.")
else:
    print("نتیجه پیش‌بینی: این کارگر در معرض خطر بیماری قرار ندارد.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step

خروجی خام مدل (احتمال بیماری): 0.5676
نتیجه پیش‌بینی: این کارگر در معرض خطر بیماری قرار دارد.


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
